# L10b: Understanding Boltzmann Machines

___
In this lab, we will explore some questions surrounding the training of a _small_ Boltzmann machine. In particular, we'll look at one of the key limitations of the training approach, namely requiring convergence to a stationary distribution for each training iteration.

* _Why are we looking at this?_ The [Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine) popularized by [Prof. G. Hinton in the mid-1980s](https://en.wikipedia.org/wiki/Geoffrey_Hinton) was (jointly) awarded the [2024 Nobel Prize in Physics](https://www.nobelprize.org/prizes/physics/2024/summary/) along with [Prof. J. Hopfield](https://en.wikipedia.org/wiki/John_Hopfield). While of little practical use (because of an issue that we will discuss today), these ideas led to the development of [the restricted Boltzmann machine](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine), which has many practical applications.

## Tasks
Before we start, execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: What is a Boltzmann Machine? (15 min)__: In this task, we take a few minutes to review the architecture and training method for a [Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine). 
* __Task 2: Estimate the stationary distribution for a small system (30 min)__: In this task, we sample the dynamics of a three-state [Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine) and explore a key question: can we estimate the stationary distribution of the Boltzmann machine?

Let's get started!

In [2]:
include("Include.jl"); # Include the required packages and codes from Include.jl

___

## Task 1: What is a Boltzmann Machine?
A [Boltzmann Machine](https://en.wikipedia.org/wiki/Boltzmann_machine) consists of a set of binary units (neurons, nodes, vertices, etc.) that are fully connected, with no self-connections. Formally, [a Boltzmann Machine](https://en.wikipedia.org/wiki/Boltzmann_machine) $\mathcal{B}$ is an fully connected _undirected weighted graph_ defined by the tuple $\mathcal{B} = \left(\mathcal{V},\mathcal{E}, \mathbf{W},\mathbf{b}, \mathbf{s}\right)$.
* __Units__: Each unit (vertex, node, neuron) $v_{i}\in\mathcal{V}$ has a binary state (`on` or `off`) and a bias value 
$b_{i}\in\mathbb{R}$, where $b_{i}$ is the bias of the node $v_{i}$. The bias vector $\mathbf{b}\in\mathbb{R}^{|\mathcal{V}|}$ is the vector of bias values for all nodes in the network. 
* __Edges__: Each edge $e\in\mathcal{E}$ has a weight. The weight of the edge connecting $v_{i}\in\mathcal{V}$ and $v_{j}\in\mathcal{V}$, is denoted by $w_{ij}\in\mathbf{W}$, where the weight matrix $\mathbf{W}\in\mathbb{R}^{|\mathcal{V}|\times|\mathcal{V}|}$ is symmetric, i.e. $w_{ij} = w_{ji}$ and $w_{ii} = 0$ (no self loops). The weights $w_{ij}\in\mathbb{R}$ determine the strength of the connection between the two nodes. 
* __States__: The state of each node is represented by a binary vector $\mathbf{s}\in\mathbb{R}^{|\mathcal{V}|}$, where $s_{i}\in\{-1,1\}$ is the state of node $v_{i}$. When $s_{i} = 1$, the node is `on`, and when $s_{i} = -1$, the node is `off`. The set of all possible state _configurations_ is denoted by $\mathcal{S} \equiv \left\{\mathbf{s}^{(1)},\mathbf{s}^{(2)},\ldots,\mathbf{s}^{(N)}\right\}$, where $N$ is the number of possible state configurations, or $N = 2^{|\mathcal{V}|}$ for binary units.

### Training Algorithm
Suppose have a collection of patterns $\mathbf{X} = \left\{\mathbf{x}^{(1)},\mathbf{x}^{(2)},\ldots,\mathbf{x}^{(m)}\right\}$, where $\mathbf{x}^{(i)}\in\mathbb{R}^{|\mathcal{V}|}$ is a binary vector of size $|\mathcal{V}|$ and $m$ is the number of patterns. The training algorithm for the [Boltzmann Machine](https://en.wikipedia.org/wiki/Boltzmann_machine) maximizes the log-likelihood of observing the training patterns $\mathbf{x}_{i}\in\mathbf{X}$ given the weights $\mathbf{W}$ and biases $\mathbf{b}$ of the network. One approach to solve this problem is _Gradient ascent_.

__Initialize__: the weights $\mathbf{W}$ and biases $\mathbf{b}$ of the network to some initial guess, e.g., [using the Hopfield network Hebbian learning rule](https://en.wikipedia.org/wiki/Hopfield_network). Set the learning rate $\eta$, temperature $\beta = 1$, and number of turns $T$. Precompute the data-dependent expectation $\langle{x_{i}x_{j}}\rangle_{\mathbf{X}}$ and $\langle{x_{i}}\rangle_{\mathbf{X}}$ using every training pattern $\mathbf{x}^{(i)}\in\mathbf{X}$.

1. Simulate the Boltzmann Machine $\mathcal{B}$ for a fixed number of turns $T$ (or until it becomes stationary). Then, generate a set of _stationary_ samples $\mathbf{S} = \left\{\mathbf{s}^{(1)},\mathbf{s}^{(2)},\ldots,\mathbf{s}^{(m)}\right\}$, then compute the model-dependent expectation $\langle{s_{i}s_{j}}\rangle_{\mathbf{S}}$ using the stationary samples $\mathbf{s}^{(i)}\in\mathbf{S}$.
    * __This step is hard__: Running the sampling until we hit the stationary distribution is challenging, and we have to do this for each iteration of the training algorithm. This is a significant issue!
3. Update the weights of the network using the following update rule: $w_{ij}^{\prime} = w_{ij} + \Delta{w_{ij}}$ where $\Delta{w_{ij}} = \eta\left(\langle{x_{i}x_{j}}\rangle_{\mathbf{X}} - \langle{s_{i}s_{j}}\rangle_{\mathbf{S}}\right)$. The hyperparameter $\eta$ is the learning rate, $\langle{x_{i}x_{j}}\rangle_{\mathbf{X}}$ is the data-dependent expectation, and $\langle{s_{i}s_{j}}\rangle_{\mathbf{S}}$ is the model-dependent expectation. The update rule is applied to all weights in the network, i.e., $\forall i,j\in\mathcal{V}$.
4. Update the biases of the network using the following update rule: $b_{i}^{\prime} = b_{i} + \Delta{b_{i}}$ where $\Delta{b_{i}} = \eta\left(\langle{x_{i}}\rangle_{\mathbf{X}} - \langle{s_{i}}\rangle_{\mathbf{S}}\right)$. The hyperparameter $\eta$ is the learning rate, $\langle{x_{i}}\rangle_{\mathbf{X}}$ is the data-dependent expectation, and $\langle{s_{i}}\rangle_{\mathbf{S}}$ is the model-dependent expectation. The update rule is applied to all biases in the network, i.e., $\forall i\in\mathcal{V}$.
5. Repeat steps 1-4 until convergence, i.e., the difference in the weight matrix $\mathbf{W}$ between subsequent iterations is small $\lVert\mathbf{W}_{i+1} - \mathbf{W}_{i}\rVert\leq\epsilon$ (or for a fixed number of iterations). 

## Task 2: Estimate the stationary distribution for a small system
In this task, we sample the dynamics of a three-state [Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine) and explore a key question: can we estimate the stationary distribution of the Boltzmann machine? We'll do this with a straightforward three-node Boltzmann machine (small enough to compute all the configurations required by the partition function). 

First, let's set up our model of the Boltzmann machine with some random parameters that we will learn in the next task. We'll save the random weights in the `W::Array{Float64,2}` matrix and the random biases in the `b::Array{Float64,1}` vector.

In [6]:
W,b = let 

    number_of_nodes = 3; # number of nodes in the system
    
    # initialize some random weights and biases
    W = 2*randn(number_of_nodes, number_of_nodes);
    b = randn(number_of_nodes);

    # subract the mean from the weights (no self connections)
    W = W - diagm(diag(W));

    # return -
    W, b
end;

__Model__: Next, let's build a model of the test Boltzmann machine. We'll use [the `MySimpleBoltzmannMachineModel` struct](src/Types.jl) to represent the machine; we build an instance of this type [using a `build(...)` method](src/Factory.jl). The struct will have `W::Array{Float64,2}` and `b::Array{Float64,1}` fields that we set when we build an instance of the model.

In [8]:
model = build(MySimpleBoltzmannMachineModel, (
    W = W,
    b = b,
));

__Constants__: Set some constants that we will use later. Please look at the comments in the code for more details on each constant's permissible values, units, etc. 

In [111]:
number_of_nodes = 3; # number of nodes in the system
β = 10.0; # temperature parameter for the system
number_of_turns = 10000; # number of turns that we take in the simulation
η = 0.01; # learning rate for the system

### Sample the test model
One of the key theoretical ideas of [the Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine) is that the samples generated from it are distributed according to [the Boltzmann distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution). Let's test this idea. 

To generate samples from a Boltzmann Machine, let us consider the following algorithm: 

__Initialize__ the weights $\mathbf{W}$ and biases $\mathbf{b}$ of the Boltzmann Machine. Provide an initial state $\mathbf{s}^{(0)}$ of the network, and a system temperature $\beta$.

For each turn $t=1,2,\dots,T$:
1. For each node $v_{i}\in\mathcal{V}$:
    1. Compute the total input $h_{i}^{(t)}$ to node $v_{i}$ using $h_{i}^{(t)} = \sum_{j\in\mathcal{V}} w_{ij}s_{j}^{(t-1)} + b_{i}$.
    2. Compute the probability of the _next_ state $s_{i}^{(t)} = 1$ using the logistic function $P(s_{i}^{(t)} = 1|h_{i}^{(t)}) = \left(1+\exp(-\beta\cdot{h}_{i}^{(t)})\right)^{-1}$ for node $v_{i}$. The probability of $s_{i}^{(t)} = -1$ is given by $P(s_{i}^{(t)} = -1|h_{i}^{(t)}) = 1 - P(s_{i}^{(t)} = 1|h_{i}^{(t)})$.
    3. Sample the _next_ state of node $v_{i}$ from a [Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) with parameter $p = P(s_{i}^{(t)} = 1|h_{i}^{(t)})$.
2. Store the state vector $\mathbf{s}^{(t)}$ of the network at turn $t$, and proceed to the next turn.

We've implemented this algorithm in [the `simulate(...)` method](src/Compute.jl).  
* The [`simulate(...)` method](src/Compute.jl) takes a [`MySimpleBoltzmannMachineModel` instance](src/Types.jl), an initial state vector `sₒ::Array{Int,1}`,
the number of turns `T::Int`, and a system (inverse) temperature `β::Float64`. The method returns an array of samples `S::Array{Int,2}` of size `N` $\times$ `T,` where `N` is the number of nodes in the Boltzmann machine and `T` is the number of turns. 

In [141]:
S = let

    # initialize the system
    N = 2^number_of_nodes; # how many configurations do we have
    energy_state = zeros(N); # energy of each state
    for i ∈ 0:(N - 1)
        sᵢ = digits(i, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1 |> reverse # Hmmm. That seems intense. What????
        energy_state[i + 1] = energy(model, sᵢ); # calculate the energy of each state
    end
    start_energy_state = argmin(energy_state); # find the state with the minimum energy
    sₒ = digits(start_energy_state - 1, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1 |> reverse # convert to -1,1
    S = simulate(model, sₒ, T = number_of_turns, β = β); # simulate the model 

    # return the data (we don't need the turn vector)``
    S;
end;

In [142]:
i = 7;
sᵢ = digits(i, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1 |> reverse

3-element Vector{Int64}:
 1
 1
 1

In [143]:
S

3×10000 Matrix{Int64}:
  1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
 -1  -1  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1

### Discussion
* __DQ1__: In the code block above, we have used a particular heuristic to select the initial state for the sampling. Could you brainstorm with your neighbor about an alternative approach? I can think of (at least) two different ways to do this, each with potentially interesting consequences.
* __DQ2__: What is the story with [the `digits(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.ndigits) - what is the line of code doing? Hint: we are counting in [base 2](https://en.wikipedia.org/wiki/Binary_number).

### What is the stationary distribution?
After a _sufficiently large_ number of turns, the network configurations (state vectors) $\mathbf{s}^{(1)},\mathbf{s}^{(2)},\dots,$ of the Boltzmann Machine will converge to a _stationary distribution_ over the state configurations $\mathbf{s}\in\mathcal{S}$ which can be modeled as [a Boltzmann distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution) of the form:
$$
P(\mathbf{s}) = \frac{1}{Z(\mathcal{S},\beta)}\exp\left(-\beta\cdot{E(\mathbf{s})}\right)
$$
where $E(\mathbf{s})$ is the energy of state $\mathbf{s}$, the $\beta$ is the (inverse) temperature of the system, and $Z(\mathcal{S},\beta)$ is the partition function. The energy of configuration $\mathbf{s}\in\mathcal{S}$ is given by:
$$
E(\mathbf{s}) = -\sum_{i\in\mathcal{V}} b_{i}s_{i} - \frac{1}{2}\sum_{i,j\in\mathcal{V}} w_{ij}s_{i}s_{j}
$$
where the first term is the energy associated with the bias terms, and the second term is the energy associated with the weights of the connections. The partition function $Z(\mathcal{S},\beta)$ is difficult to compute in practice; however, it is given by:
$$
Z(\mathcal{S},\beta) = \sum_{\mathbf{s}^{\prime}\in\mathcal{S}}\exp\left({-\beta\cdot{E}(\mathbf{s}^{\prime})}\right)
$$
where $\mathcal{S}$ is the set of _all possible network configurations_ of the Boltzmann Machine. 
* __Hmmm...__? The partition function $Z(\mathcal{S},\beta)$ is a normalizing constant that ensures that the probabilities sum to 1. However, for even a moderately sized system, the partition function is impossible to compute because it involves summing over all possible network configurations, which grows exponentially with the number of nodes.
* For example, in our case, that is $2^{n}$, where $n$ is the number of nodes in the network. For our simple three-node Boltzmann machine, the partition function will sum $2^{3} = 8$ states. We can enumerate these states using [the `digits(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.ndigits).

In [146]:
Z,configurations = let

    # initialize -
    Z = Dict{Int,Float64}();
    configurations = Dict{Int,Vector{Int}}();
    N = 2^number_of_nodes; # how many configurations do we have

    # loop throught each configuration
    for i ∈ 0:(N - 1)
        sᵢ = digits(i, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1 |> reverse # wow! that is really nice.
        Z[i] = exp(-2*β*energy(model, sᵢ)); # calculate the partition function
        configurations[i] = sᵢ; # store the configuration
    end

    # return -
    Z,configurations
end;

__Compute the _actual_ stationary distribution__: Let's compute the stationary distribution of the Boltzmann Machine using the Boltzmann distribution. We'll compute the energy of each state configuration $\mathbf{s}\in\mathcal{S}$ and then compute the probability of each state configuration using the Boltzmann distribution. 

We'll save the probabilities in the `P::Dict{Int,Array{Float64,1}}` dictionary where the key is the state configuration and the value is the probability of the state configuration.

In [148]:
P = let
    
    # initialize -
    P = Dict{Int,Float64}();
    N = 2^number_of_nodes; # how many configurations do we have

    # what is the normalizing constant
    Z̄ = sum(values(Z)); # calculate the value of the partition function

    # loop through each configuration
    for i ∈ 0:(N - 1)
        P[i] = Z[i]/Z̄; # calculate the probability of each configuration
    end

    # return -
    P
end;

__Check__: Does the _actual_ stationary Boltzmann distribution sum to `1` (use [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert)? If not, then we have a problem.

In [150]:
@assert sum(values(P)) ≈ 1.0 # if this fails: we get an AssertionError, otherwise nothing happens

Estimate the empirical stationary distribution: Next, compute the empirical estimate of the stationary distribution by analyzing the simulation samples. If we generate enough samples, the empirical distribution should be similar to the stationary distribution. 
* __Idea__: Compute the number of times a particular configuration $\mathbf{s}\in\mathcal{S}$ occurs in the simulation sample matrix $\mathbf{S}$ for each of the configurations, and then divide by the total number of samples to get the probability of each configuration. This gives us the _emphirical_ distribution of the samples.

We'll save the empirical probabilities in the `P̂::Dict{Int, Array{Float64,1}}` dictionary, where the key is the state configuration index and the value is the empirical probability of that state configuration.

In [152]:
P̂ = let
   
    # initialize -
    P̂ = Dict{Int,Float64}();
    N = 2^number_of_nodes; # how many configurations do we have
    number_of_turns = size(S,2); # how many turns do we have

    for i ∈ 0:(N - 1)
        sᵢ = digits(i, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1 |> reverse # count by base 2, and convert to -1,1

        counter = 0;
        for j ∈ 1:number_of_turns
            if (S[:,j] == sᵢ)
                counter += 1;
            end
        end
        P̂[i] = counter/number_of_turns;
    end
    
    P̂
end;

__Check__: Does the emphirical stationary Boltzmann distribution sum to `1` (use [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert)? If not, then we have a problem.

In [154]:
@assert sum(values(P̂)) ≈ 1.0 # if this fails: we get an AssertionError, otherwise nothing happens

`Unhide` the code block below to see how we constructed the probability table for the actual and emphirical stationary distributions.

In [156]:
let
   
    # initialize -
    df = DataFrame();
    N = 2^number_of_nodes; # how many configurations do we have

    # compute the ordinal rank -
    r = [P[i] for i ∈ 0:(N - 1)] |> x -> ordinalrank(x, rev = true);
    r̂ = [P̂[i] for i ∈ 0:(N - 1)] |> x -> ordinalrank(x, rev = true);

    # main -
    for i ∈ 0:(N - 1)
        sᵢ = digits(i, base = 2, pad = number_of_nodes) |> x -> 2*x .- 1|> reverse # count by base 2, and convert to -1,1
        row_df = (
            i = i,
            configuration = sᵢ,
            energy = energy(model, sᵢ),
            P = P[i],
            P̂ = P̂[i],
            r = r[i+1],
            r̂ = r̂[i+1],
        )
        push!(df, row_df)
    end
    
    pretty_table(df, tf = tf_simple);
end

======== =============== ========== ============= ========= ======= ========
      i   configuration     energy             P         P̂       r       r̂ 
  Int64   Vector{Int64}    Float64       Float64   Float64   Int64   Int64 
======== =============== ========== ============= ========= ======= ========
      0    [-1, -1, -1]    3.22406   7.81212e-67       0.0       6       2
      1     [-1, -1, 1]    3.59841   4.37715e-70       0.0       7       3
      2     [-1, 1, -1]   -3.69118    9.07425e-7       0.0       3       4
      3      [-1, 1, 1]   -4.08754    0.00251516       0.0       2       5
      4     [1, -1, -1]   -4.38669      0.997484       1.0       1       1
      5      [1, -1, 1]   0.503239   3.35381e-43       0.0       5       6
      6      [1, 1, -1]   0.360242   5.85586e-42       0.0       4       7
      7       [1, 1, 1]    4.47946   9.73992e-78       0.0       8       8
======== =============== ========== ============= ========= ======= ========


### Discussion
* __DQ1__: What happens to the probability of the states as we change the system (inverse) temperature $\beta$, i.e., do we see different behavior for (cool) $\beta\gg{1}$ versus (hot) $\beta\ll{1}$ systems?
* __DQ2__: In the gradient ascent training algorithm, the new step is given by: $\Delta{w_{ij}} = \eta\left(\langle{x_{i}x_{j}}\rangle_{\mathbf{X}} - \langle{s_{i}s_{j}}\rangle_{\mathbf{S}}\right)$. What is you interpolation of this, and how would we compute this update?

## Next time
In lecture `L10c` we will continue our discussion of Boltzmann machines, but will focus on a more practical implememtation called [the Restricted Boltzmann Machine](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine). 

* The RBM is a special case of the Boltzmann machine where the nodes are divided into two layers: a visible layer and a hidden layer. The visible layer contains the input data, and the hidden layer contains the latent variables. The RBM is trained [using the contrastive divergence algorithm](https://www.cs.toronto.edu/~fritz/absps/cdmiguel.pdf).